# Common Security Vulnerabilities and Best Practices

SQL Injection:

SQL injection occurs when an attacker is able to inject arbitrary SQL code into a query.

Always use parameterized queries or prepared statements to prevent SQL injection.

In [ ]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('vehicles.db')
cursor = conn.cursor()

# Secure function to add a vehicle using parameterized queries
def add_vehicle(make, model, year, price, manufacturer_id, owner_id):
    cursor.execute('''
    INSERT INTO vehicles (make, model, year, price, manufacturer_id, owner_id)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (make, model, year, price, manufacturer_id, owner_id))
    conn.commit()
    return cursor.lastrowid

# Example
manufacturer_id = 1
owner_id = 1
vehicle_id = add_vehicle('Toyota', 'Corolla', 2020, 20000, manufacturer_id, owner_id)


Cross-Site Scripting (XSS):

XSS occurs when an attacker injects malicious scripts into content from otherwise trusted websites.

If using a web framework like Flask, use templates to escape output.

In [ ]:
from flask import Flask, render_template_string, request

app = Flask(__name__)

# Example route with input sanitization
@app.route('/greet', methods=['GET'])
def greet():
    name = request.args.get('name', 'World')
    safe_name = escape(name)  # Escape user input to prevent XSS
    return render_template_string('<h1>Hello, {{ safe_name }}!</h1>', safe_name=safe_name)

if __name__ == '__main__':
    app.run()


# Input Validation and Sanitization

Validating and sanitizing inputs ensures that the data received by your application is correct and safe to use

In [ ]:
import re
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('vehicles.db')
cursor = conn.cursor()

# Function to validate vehicle inputs
def validate_vehicle(make, model, year, price):
    if not re.match(r'^[a-zA-Z0-9\s]+$', make):
        raise ValueError('Invalid make')
    if not re.match(r'^[a-zA-Z0-9\s]+$', model):
        raise ValueError('Invalid model')
    if not isinstance(year, int) or year < 1886 or year > 2023:  # Assuming valid car years
        raise ValueError('Invalid year')
    if not isinstance(price, (int, float)) or price < 0:
        raise ValueError('Invalid price')

# Secure and validated function to add a vehicle
def add_vehicle(make, model, year, price, manufacturer_id, owner_id):
    validate_vehicle(make, model, year, price)  # Validate inputs
    cursor.execute('''
    INSERT INTO vehicles (make, model, year, price, manufacturer_id, owner_id)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (make, model, year, price, manufacturer_id, owner_id))
    conn.commit()
    return cursor.lastrowid

# Example
manufacturer_id = 1
owner_id = 1
try:
    vehicle_id = add_vehicle('Toyota', 'Corolla', 2020, 20000, manufacturer_id, owner_id)
except ValueError as e:
    print(f"Input error: {e}")


# Authentication and Authorization Mechanisms

Authentication: Verifying the identity of a user.

Authorization: Determining whether a user has permission to perform a certain action.

In [ ]:
from flask import Flask, request, jsonify, abort
from functools import wraps

app = Flask(__name__)

# Dummy data for users
users = {
    'admin': 'password123',
    'user1': 'mypassword'
}

# Dummy roles
roles = {
    'admin': ['admin'],
    'user1': ['user']
}

# Authentication decorator
def check_auth(username, password):
    return username in users and users[username] == password

def authenticate():
    return abort(401, 'Unauthorized')

def requires_auth(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        auth = request.authorization
        if not auth or not check_auth(auth.username, auth.password):
            return authenticate()
        return f(*args, **kwargs)
    return decorated

# Authorization decorator
def requires_role(role):
    def decorator(f):
        @wraps(f)
        def decorated(*args, **kwargs):
            auth = request.authorization
            if not auth or not check_auth(auth.username, auth.password):
                return authenticate()
            if role not in roles.get(auth.username, []):
                return abort(403, 'Forbidden')
            return f(*args, **kwargs)
        return decorated
    return decorator

@app.route('/vehicles', methods=['GET'])
@requires_auth
def get_vehicles():
    # Retrieve vehicles logic here
    return jsonify({"vehicles": []})

@app.route('/admin/vehicles', methods=['GET'])
@requires_auth
@requires_role('admin')
def admin_get_vehicles():
    # Admin-only retrieve vehicles logic here
    return jsonify({"vehicles": []})

if __name__ == '__main__':
    app.run()
